# Test task

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("test_data.csv")
df_c = df.copy(deep = True)

In [3]:
#оставляем только реплики менеджеров
df_manager = df_c.drop(df_c[df_c.role == "client"].index)
df_manager

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
11,0,11,manager,Что для вас приоритет
...,...,...,...,...
472,5,135,manager,Так дмитрий
473,5,136,manager,Все записала тогда завтра ждите звонка
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...


### Реплики приветсвия

In [4]:
greetings = []
searchfor = ['дравствуйте', 'обрый день']
greetings.append(df_manager[df_manager['text'].str.contains('|'.join(searchfor))].text)

greetings

[1             Алло здравствуйте
 110           Алло здравствуйте
 166           Алло здравствуйте
 250    Алло дмитрий добрый день
 Name: text, dtype: object]

### Менеджеры представляются

In [5]:
introduced = []
introduced.append(df_manager[df_manager['text'].str.contains('зовут')].text)

introduced 

[3      Меня зовут ангелина компания диджитал бизнес з...
 111    Меня зовут ангелина компания диджитал бизнес з...
 167    Меня зовут ангелина компания диджитал бизнес з...
 251    Добрый меня максим зовут компания китобизнес у...
 Name: text, dtype: object]

In [6]:
import re
import numpy as np

In [7]:
introduced = np.concatenate(introduced).astype(str)
introduced

array(['Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
       'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
       'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
       'Добрый меня максим зовут компания китобизнес удобно говорить'],
      dtype='<U131')

In [8]:
names = [re.findall('(?<=еня зовут )\w+', s) or re.findall(r'меня (.*?) зовут', s) for s in introduced] 
names

[['ангелина'], ['ангелина'], ['ангелина'], ['максим']]

### Названия компаний

In [9]:
companies = []
companies.append(df_manager[df_manager['text'].str.contains('омпания')].text)

companies

[3      Меня зовут ангелина компания диджитал бизнес з...
 111    Меня зовут ангелина компания диджитал бизнес з...
 167    Меня зовут ангелина компания диджитал бизнес з...
 251    Добрый меня максим зовут компания китобизнес у...
 273    Транспортная компания хорошо а у вас уже вот в...
 Name: text, dtype: object]

In [10]:
companies_names = [re.findall('(?<=омпания )\w+', s) for s in np.concatenate(companies).astype(str)]
companies_names

[['диджитал'], ['диджитал'], ['диджитал'], ['китобизнес'], ['хорошо']]

### Реплики прощания

In [11]:
goodbyes = []
searchfor = ['о свидания', 'сего доброго']
goodbyes.append(df_manager[df_manager['text'].str.contains('|'.join(searchfor))].text)

goodbyes

[108                           Всего хорошего до свидания
 163                                          До свидания
 300    Угу все хорошо да понедельника тогда всего доб...
 335    Во вторник все ну с вами да тогда до вторника ...
 479                       Ну до свидания хорошего вечера
 Name: text, dtype: object]

### Проверка требований к менеджеру

In [12]:
text_by_mangers = df_manager.groupby('dlg_id').agg(', '.join).drop(labels='role', axis=1)
text_by_mangers['greet_gbyes'] = text_by_mangers['text'].str.contains('|'.join(np.concatenate(greetings).astype(str))) & text_by_mangers['text'].str.contains('|'.join(np.concatenate(goodbyes).astype(str)))
text_by_mangers

,text,greet_gbyes
dlg_id,,
0,"Алло здравствуйте, Меня зовут ангелина компани...",True
1,"Алло здравствуйте, Меня зовут ангелина компани...",True
2,"Алло здравствуйте, Меня зовут ангелина компани...",False
3,"Алло дмитрий добрый день, Добрый меня максим з...",True
4,Вот по виду платежи пообщаться помните мы вот ...,False
5,"Да это анастасия, Угу, Так да да хорошо смотри...",False
